In [ ]:
# getting mocap data
from support.funcs import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df_mocap,starttime=read_df_csv(r'C:\Users\arpan\OneDrive\Documents\internship\mocapcsv\arpan_t2.csv')

# converting to epoch time
epoch_time = starttime.timestamp() 
time=df_mocap['seconds'].to_numpy()
time=time+epoch_time
df_mocap.insert(1,'epoch_time',time)
df_mocap.columns

# getting mpipe data
df_mpipe=pd.read_csv('mpipe.csv')
df_mpipe


In [ ]:
# finding calibration frame
f=False
for ind,val in enumerate(df_mpipe['epoch_time'].tolist()):
    for i,v in enumerate(df_mocap['epoch_time'].tolist()):
        if v==val:
            print(ind,i,val)
            mpipe_cf=ind
            mocap_cf=i
            f=True
            break
    if f:
        break

print('\n-----------------------\n')

for ind,val in enumerate(df_mpipe['epoch_time'].tolist()):
    for i,v in enumerate(df_mocap['epoch_time'].tolist()):
        if v==val:
            print(ind,i,val)

In [ ]:
# calibrate the frame shoulder frame of reference

# mpipe_cf=20

mpiperrf=find_orthogonal_frame([df_mpipe['RK_x'][mpipe_cf]-df_mpipe['RH_x'][mpipe_cf],df_mpipe['RK_y'][mpipe_cf]-df_mpipe['RH_y'][mpipe_cf],df_mpipe['RK_z'][mpipe_cf]-df_mpipe['RH_z'][mpipe_cf]],
                               [df_mpipe['RA_x'][mpipe_cf]-df_mpipe['RK_x'][mpipe_cf],df_mpipe['RA_y'][mpipe_cf]-df_mpipe['RK_y'][mpipe_cf],df_mpipe['RA_z'][mpipe_cf]-df_mpipe['RK_z'][mpipe_cf]])

mpipelrf=find_orthogonal_frame([df_mpipe['LK_x'][mpipe_cf]-df_mpipe['LH_x'][mpipe_cf],df_mpipe['LK_y'][mpipe_cf]-df_mpipe['LH_y'][mpipe_cf],df_mpipe['LK_z'][mpipe_cf]-df_mpipe['LH_z'][mpipe_cf]],
                               [df_mpipe['LA_x'][mpipe_cf]-df_mpipe['LK_x'][mpipe_cf],df_mpipe['LA_y'][mpipe_cf]-df_mpipe['LK_y'][mpipe_cf],df_mpipe['LA_z'][mpipe_cf]-df_mpipe['LK_z'][mpipe_cf]])

# mocaprrf=find_orthogonal_frame([df_mocap['RK_x'][mocap_cf]-df_mocap['RH_x'][mocap_cf],df_mocap['RK_y'][mocap_cf]-df_mocap['RH_y'][mocap_cf],df_mocap['RK_z'][mocap_cf]-df_mocap['RH_z'][mocap_cf]],
#                                [df_mocap['RA_x'][mocap_cf]-df_mocap['RK_x'][mocap_cf],df_mocap['RA_y'][mocap_cf]-df_mocap['RK_y'][mocap_cf],df_mocap['RA_z'][mocap_cf]-df_mocap['RK_z'][mocap_cf]])

# mocaplrf=find_orthogonal_frame([df_mocap['LK_x'][mocap_cf]-df_mocap['LH_x'][mocap_cf],df_mocap['LK_y'][mocap_cf]-df_mocap['LH_y'][mocap_cf],df_mocap['LK_z'][mocap_cf]-df_mocap['LH_z'][mocap_cf]],
#                                [df_mocap['LA_x'][mocap_cf]-df_mocap['LK_x'][mocap_cf],df_mocap['LA_y'][mocap_cf]-df_mocap['LK_y'][mocap_cf],df_mocap['LA_z'][mocap_cf]-df_mocap['LK_z'][mocap_cf]])


In [ ]:
#calculating the knee angle
mpipeRKa, mocapRKa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeRKa.append(angle3point([df_mpipe['RH_x'][i],df_mpipe['RH_y'][i],df_mpipe['RH_z'][i]],
                                [df_mpipe['RK_x'][i],df_mpipe['RK_y'][i],df_mpipe['RK_z'][i]],
                                [df_mpipe['RA_x'][i],df_mpipe['RA_y'][i],df_mpipe['RA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapRKa.append(angle3point([df_mocap['RH_x'][i],df_mocap['RH_y'][i],df_mocap['RH_z'][i]],
#                                 [df_mocap['RK_x'][i],df_mocap['RK_y'][i],df_mocap['RK_z'][i]],
#                                 [df_mocap['RA_x'][i],df_mocap['RA_y'][i],df_mocap['RA_z'][i]]))
    
#test graph right elbow angle wihtout occlusion
pvt(mpipeRKa,df_mpipe['epoch_time'])
# pvt(mocapRKa,df_mocap['epoch_time'])
plt.title('Right knee Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
# plt.ylim(0,180)
plt.show()

# print('RMSE of right knee Angle:',RMSE(mocapRKa,mpipeRKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

mpipeLKa, mocapLKa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeLKa.append(angle3point([df_mpipe['LH_x'][i],df_mpipe['LH_y'][i],df_mpipe['LH_z'][i]],
                                [df_mpipe['LK_x'][i],df_mpipe['LK_y'][i],df_mpipe['LK_z'][i]],
                                [df_mpipe['LA_x'][i],df_mpipe['LA_y'][i],df_mpipe['LA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapLKa.append(angle3point([df_mocap['LH_x'][i],df_mocap['LH_y'][i],df_mocap['LH_z'][i]],
#                                 [df_mocap['LK_x'][i],df_mocap['LK_y'][i],df_mocap['LK_z'][i]],
#                                 [df_mocap['LA_x'][i],df_mocap['LA_y'][i],df_mocap['LA_z'][i]]))
   
#test graph right elbow angle wihtout occlusion
pvt(mpipeLKa,df_mpipe['epoch_time'])
# pvt(mocapLKa,df_mocap['epoch_time'])
plt.title('Left knee Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
# plt.ylim(0,180)
plt.show()
# print('RMSE of left knee Angle:',RMSE(mocapLKa,mpipeLKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))


In [ ]:
#calculating the ankle angle
mpipeRAa, mocapRaa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeRAa.append(angle3point([df_mpipe['RT_x'][i],df_mpipe['RT_y'][i],df_mpipe['RT_z'][i]],
                                [df_mpipe['RK_x'][i],df_mpipe['RK_y'][i],df_mpipe['RK_z'][i]],
                                [df_mpipe['RA_x'][i],df_mpipe['RA_y'][i],df_mpipe['RA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapRAa.append(angle3point([df_mocap['RT_x'][i],df_mocap['RT_y'][i],df_mocap['RT_z'][i]],
#                                 [df_mocap['RK_x'][i],df_mocap['RK_y'][i],df_mocap['RK_z'][i]],
#                                 [df_mocap['RA_x'][i],df_mocap['RA_y'][i],df_mocap['RA_z'][i]]))
    
#test graph right elbow angle wihtout occlusion
pvt(mpipeRAa,df_mpipe['epoch_time'])
# pvt(mocapRAa,df_mocap['epoch_time'])
plt.title('Right ankle Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
plt.ylim(0,180)
plt.show()

# print('RMSE of right knee Angle:',RMSE(mocapRKa,mpipeRKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

mpipeLAa, mocapLAa=[],[]

for i in range(len(df_mpipe['epoch_time'].tolist())):
    mpipeLAa.append(angle3point([df_mpipe['LT_x'][i],df_mpipe['LT_y'][i],df_mpipe['LT_z'][i]],
                                [df_mpipe['LK_x'][i],df_mpipe['LK_y'][i],df_mpipe['LK_z'][i]],
                                [df_mpipe['LA_x'][i],df_mpipe['LA_y'][i],df_mpipe['LA_z'][i]]))
   
# for i in range(len(df_mocap['epoch_time'].tolist())):
#     mocapLAa.append(angle3point([df_mocap['LT_x'][i],df_mocap['LT_y'][i],df_mocap['LT_z'][i]],
#                                 [df_mocap['LK_x'][i],df_mocap['LK_y'][i],df_mocap['LK_z'][i]],
#                                 [df_mocap['LA_x'][i],df_mocap['LA_y'][i],df_mocap['LA_z'][i]]))
   
#test graph right elbow angle wihtout occlusion
print(mpipeLAa)
pvt(mpipeLAa,df_mpipe['epoch_time'])
print('1')
# pvt(mocapLAa,df_mocap['epoch_time'])
plt.title('Left ankle Angle in degrees ')
plt.xlabel('time(s)')
plt.ylabel('Angle(degrees)')
# plt.legend(['mediapipe','mocap'])
# plt.ylim(0,180)
plt.show()
# print('RMSE of left knee Angle:',RMSE(mocapLKa,mpipeLKa,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

In [ ]:
print(mpipeLAa)
print(mpipeLKa)

In [ ]:
# calculating euler angles for hip xzy-flexion/extension,abduction/adduction,internal/external rotation

mpipertheta1,mpipertheta2,mpipertheta3=[],[],[]
mpipeltheta1,mpipeltheta2,mpipeltheta3=[],[],[]
kath=160
for i in range(len(df_mpipe['epoch_time'].tolist())):
    newmat=find_orthogonal_frame([df_mpipe['RK_x'][i]-df_mpipe['RH_x'][i],df_mpipe['RK_y'][i]-df_mpipe['RH_y'][i],df_mpipe['RK_z'][i]-df_mpipe['RH_z'][i]],
                                 [df_mpipe['RA_x'][i]-df_mpipe['RK_x'][i],df_mpipe['RA_y'][i]-df_mpipe['RK_y'][i],df_mpipe['RA_z'][i]-df_mpipe['RK_z'][i]])
    rotmat=find_rotation_matrix(mpiperrf,newmat)
    mpiperthetas=rotation_angles(rotmat)

    newmat=find_orthogonal_frame([df_mpipe['LK_x'][mpipe_cf]-df_mpipe['LH_x'][mpipe_cf],df_mpipe['LK_y'][mpipe_cf]-df_mpipe['LH_y'][mpipe_cf],df_mpipe['LK_z'][mpipe_cf]-df_mpipe['LH_z'][mpipe_cf]],
                                 [df_mpipe['LA_x'][mpipe_cf]-df_mpipe['LK_x'][mpipe_cf],df_mpipe['LA_y'][mpipe_cf]-df_mpipe['LK_y'][mpipe_cf],df_mpipe['LA_z'][mpipe_cf]-df_mpipe['LK_z'][mpipe_cf]])
    rotmat=find_rotation_matrix(mpipelrf,newmat)
    mpipelthetas=rotation_angles(rotmat)

    mpipertheta1.append(mpiperthetas[0])
    mpipertheta2.append(mpiperthetas[1])
    if mpipeRKa[i]<kath:
        mpipertheta3.append(-mpiperthetas[2])  
    else:
        mpipertheta3.append(np.nan)

    mpipeltheta1.append(mpipelthetas[0])
    mpipeltheta2.append(-mpipelthetas[1])
    if mpipeLKa[i]<kath:
        mpipeltheta3.append(mpipelthetas[2])  
    else:
        mpipeltheta3.append(np.nan)

mocaprtheta1,mocaprtheta2,mocaprtheta3=[],[],[]
mocapltheta1,mocapltheta2,mocapltheta3=[],[],[]

# for i in range(len(df_mocap['epoch_time'].tolist())):
#     newmat=find_orthogonal_frame([df_mocap['RK_x'][i]-df_mocap['RH_x'][i],df_mocap['RK_y'][i]-df_mocap['RH_y'][i],df_mocap['RK_z'][i]-df_mocap['RH_z'][i]],
#                                  [df_mocap['RA_x'][i]-df_mocap['RK_x'][i],df_mocap['RA_y'][i]-df_mocap['RK_y'][i],df_mocap['RA_z'][i]-df_mocap['RK_z'][i]])
#     rotmat=find_rotation_matrix(mocaprrf,newmat)
#     mocaprthetas=rotation_angles(rotmat)

#     newmat=find_orthogonal_frame([df_mocap['RK_x'][mocap_cf]-df_mocap['RH_x'][mocap_cf],df_mocap['RK_y'][mocap_cf]-df_mocap['RH_y'][mocap_cf],df_mocap['RK_z'][mocap_cf]-df_mocap['RH_z'][mocap_cf]],
#                                  [df_mocap['RA_x'][mocap_cf]-df_mocap['RK_x'][mocap_cf],df_mocap['RA_y'][mocap_cf]-df_mocap['RK_y'][mocap_cf],df_mocap['RA_z'][mocap_cf]-df_mocap['RK_z'][mocap_cf]])
#     rotmat=find_rotation_matrix(mocaplrf,newmat)
#     mocaplthetas=rotation_angles(rotmat)
    
#     mocaprtheta1.append(mocaprthetas[0])
#     mocaprtheta2.append(mocaprthetas[1])
#     if mocapRKa[i]<kath:
#         mocaprtheta3.append(-mocaprthetas[2])  
#     else:
#         mocaprtheta3.append(np.nan)
#     mocapltheta1.append(mocaplthetas[0])
#     mocapltheta2.append(-mocaplthetas[1])
#     if mocapLKa[i]<kath:
#         mocapltheta3.append(mocaplthetas[2])  
#     else:
#         mocapltheta3.append(np.nan)

In [ ]:
pvt(mpipertheta1,df_mpipe['epoch_time'])
# pvt(mocaprtheta1,df_mocap['epoch_time'])
# plt.legend(['mediapipe','mocap'])
plt.title('rtheta1 flexion')
plt.ylim(-90,90)
plt.show()

# print('RMSE of flexion/extension: ',RMSE(mocaprtheta1,mpipertheta1,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

pvt(mpipertheta2,df_mpipe['epoch_time'])
# pvt(mocaprtheta2,df_mocap['epoch_time'])
# plt.legend(['mediapipe','mocap'])
plt.title('rtheta2 abduction')
plt.ylim(-90,90)
plt.show()

# print('RMSE of abduct/adduct: ',RMSE(mocaprtheta2,mpipertheta2,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

pvt(mpipertheta3,df_mpipe['epoch_time'].tolist())
# pvt(mocaprtheta3,df_mocap['epoch_time'].tolist())
# plt.legend(['mediapipe','mocap'])
plt.title('rtheta3 rotation')
plt.ylim(-90,90)
plt.show()

# print('RMSE of internal/external rotation: ',RMSE(mocaprtheta3,mpipertheta3,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

pvt(mpipeltheta1,df_mpipe['epoch_time'])
# pvt(mocapltheta1,df_mocap['epoch_time'])
# plt.legend(['mediapipe','mocap'])
plt.title('ltheta1 flexion')
plt.ylim(-90,90)
plt.show()

# print('RMSE of flexion/extension: ',RMSE(mocapltheta1,mpipeltheta1,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

pvt(mpipeltheta2,df_mpipe['epoch_time'])
# pvt(mocapltheta2,df_mocap['epoch_time'])
# plt.legend(['mediapipe','mocap'])
plt.title('ltheta2 abduction')
plt.ylim(-90,90)
plt.show()

# print('RMSE of abduct/adduct: ',RMSE(mocapltheta2,mpipeltheta2,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))

pvt(mpipeltheta3,df_mpipe['epoch_time'].tolist())
# pvt(mocapltheta3,df_mocap['epoch_time'].tolist())
# plt.legend(['mediapipe','mocap'])
plt.title('ltheta3 rotation')
plt.ylim(-90,90)
plt.show()

# print('RMSE of internal/external rotation: ',RMSE(mocapltheta3,mpipeltheta3,df_mocap['epoch_time'].to_numpy(),df_mpipe['epoch_time'].to_numpy()))